In [1]:
import os
import re

import pandas as pd
import numpy as np
import altair as alt
import datetime as dt

In [2]:
from statsmodels.tsa.seasonal import seasonal_decompose, DecomposeResult

In [3]:
msa_files = [os.path.join("msa", file_path) for file_path in os.listdir('msa')]
ZHVI_files = [os.path.join('zillow', file_path) for file_path in os.listdir( 'zillow')]

In [4]:
msa_dfs = [pd.read_excel(f, skiprows=7)[1:] for f in msa_files]
print(msa_files)

['msa/msamonthly_202102.xls', 'msa/msamonthly_202103.xls', 'msa/msamonthly_202101.xls', 'msa/msamonthly_202110.xls', 'msa/msamonthly_202104.xls', 'msa/msamonthly_202105.xls', 'msa/msamonthly_202111.xls', 'msa/msamonthly_202107.xls', 'msa/msamonthly_202112.xls', 'msa/msamonthly_202106.xls', 'msa/msamonthly_202001.xls', 'msa/msamonthly_202203.xls', 'msa/msamonthly_201911.xls', 'msa/msamonthly_202202.xls', 'msa/msamonthly_202002.xls', 'msa/msamonthly_201912.xls', 'msa/msamonthly_202201.xls', 'msa/msamonthly_202003.xls', 'msa/msamonthly_202007.xls', 'msa/msamonthly_202205.xls', 'msa/msamonthly_202204.xls', 'msa/msamonthly_202012.xls', 'msa/msamonthly_202006.xls', 'msa/msamonthly_202010.xls', 'msa/msamonthly_202004.xls', 'msa/msamonthly_202206.xls', 'msa/msamonthly_202207.xls', 'msa/msamonthly_202005.xls', 'msa/msamonthly_202011.xls', 'msa/msamonthly_202008.xls', 'msa/msamonthly_202009.xls', 'msa/msamonthly_202108.xls', 'msa/msamonthly_202109.xls']


In [5]:
print(msa_dfs[0].columns)

msa_cols = ['CSA', 'CBSA', 'Name', 'Total', '1 Unit', '2 Units', 
            '3 and 4 Units','5 Units or More', 
            'Num of Structures With 5 Units or More']
msa_dfs = [df[msa_cols] for df in msa_dfs]

msa_dfs[0].head()

Index(['CSA', 'CBSA', 'Name', 'Total', '1 Unit', '2 Units', '3 and 4 Units',
       '5 Units or More', 'Num of Structures With 5 Units or More',
       'Monthly Coverage Percent*', 'Unnamed: 10', 'Total.1', '1 Unit.1',
       '2 Units.1', '3 and 4 Units.1', '5 Units or More.1',
       'Num of Structures With 5 Units or More.1'],
      dtype='object')


,CSA,CBSA,Name,Total,1 Unit,2 Units,3 and 4 Units,5 Units or More,Num of Structures With 5 Units or More
1,999.0,10180.0,"Abilene, TX ...",33.0,31.0,2.0,0.0,0.0,0.0
2,184.0,10420.0,"Akron, OH ...",45.0,45.0,0.0,0.0,0.0,0.0
3,999.0,10500.0,"Albany, GA ...",32.0,15.0,0.0,0.0,17.0,1.0
4,440.0,10540.0,"Albany-Lebanon, OR ...",58.0,37.0,0.0,0.0,21.0,3.0
5,104.0,10580.0,"Albany-Schenectady-Troy, NY ...",268.0,118.0,4.0,0.0,146.0,2.0


In [6]:
print(msa_dfs[0].columns)

Index(['CSA', 'CBSA', 'Name', 'Total', '1 Unit', '2 Units', '3 and 4 Units',
       '5 Units or More', 'Num of Structures With 5 Units or More'],
      dtype='object')


In [7]:
for df, name in zip(msa_dfs, msa_files):
    df['Date'] = name[name.rfind('_') + 1:name.rfind('.')]

df.head()

,CSA,CBSA,Name,Total,1 Unit,2 Units,3 and 4 Units,5 Units or More,Num of Structures With 5 Units or More,Date
1,999.0,10180.0,"Abilene, TX ...",62.0,56.0,6.0,0.0,0.0,0.0,202109
2,184.0,10420.0,"Akron, OH ...",62.0,57.0,0.0,0.0,5.0,1.0,202109
3,999.0,10500.0,"Albany, GA ...",40.0,26.0,0.0,0.0,14.0,1.0,202109
4,440.0,10540.0,"Albany-Lebanon, OR ...",66.0,29.0,4.0,3.0,30.0,3.0,202109
5,104.0,10580.0,"Albany-Schenectady-Troy, NY ...",273.0,97.0,2.0,3.0,171.0,4.0,202109


In [8]:
housing_files = [
    'acs-housing/ACSDP1Y2010.DP04-2022-10-14T204332.csv',
    'acs-housing/ACSDP1Y2011.DP04-2022-10-14T204308.csv',
    'acs-housing/ACSDP1Y2012.DP04-2022-10-14T204206.csv',
    'acs-housing/ACSDP1Y2013.DP04-2022-10-14T204143.csv',
    'acs-housing/ACSDP1Y2014.DP04-2022-10-14T204119.csv',
    'acs-housing/ACSDP1Y2015.DP04-2022-10-14T204057.csv',
    'acs-housing/ACSDP1Y2016.DP04-2022-10-14T204034.csv',
    'acs-housing/ACSDP1Y2017.DP04-2022-10-14T204014.csv',
    'acs-housing/ACSDP1Y2018.DP04-2022-10-14T203953.csv',
    'acs-housing/ACSDP1Y2019.DP04-2022-10-14T203932.csv',
    'acs-housing/ACSDP5Y2020.DP04-2022-10-14T203917.csv',
    'acs-housing/ACSDP1Y2021.DP04-2022-10-14T203721.csv',
]

housing_df = pd.DataFrame()
# housing_df_list = list()
for file_path in housing_files:
    house_year_df = pd.read_csv(file_path).set_index("Label (Grouping)").transpose()
    house_year_df.columns = [col.strip() for col in house_year_df.columns]
    
    housing_columns = [
        'Total housing units', 'Occupied housing units', 'Vacant housing units', 'Homeowner vacancy rate',
        'Rental vacancy rate', 'Total housing units', '1-unit, detached',
        '1-unit, attached', '2 units', '3 or 4 units', '5 to 9 units', '10 to 19 units', '20 or more units',
        '1 room', '2 rooms', '3 rooms', '4 rooms', '5 rooms', '6 rooms', '7 rooms', '8 rooms', '9 rooms or more',
        'Median rooms', 'No bedroom', '1 bedroom', '2 bedrooms', '3 bedrooms', '4 bedrooms', '5 or more bedrooms'
    ]
    house_year_df = house_year_df[housing_columns].copy()
    
    year = re.match(r'.*ACSDP\dY(?P<year>\d{4}).*', file_path).group('year')
    # print(year, file_path)
    
    # Remove unnecessary observations
    index = [idx for idx in house_year_df.index if idx.endswith(" CSA!!Estimate")]
    house_year_df = house_year_df.loc[index]
    
    # Rename our indices to just be the CSA Name
    renamed_index = tuple(idx.replace(" CSA!!Estimate", "") for idx in house_year_df.index)
    house_year_df.index = renamed_index
        
    # Set index
    house_year_df.index.name = "CSA"
    house_year_df['Year'] = year
    house_year_df = house_year_df.reset_index().set_index(['Year', 'CSA'])
    
    # Look for intersection between our indices ...
    # print(set(housing_df.index).intersection(house_year_df.index))
    housing_df = pd.concat([housing_df, house_year_df]).copy()
    # housing_df_list.append(house_year_df)

housing_df.reset_index(inplace=True)
housing_df.head()

,Year,CSA,Total housing units,Total housing units,Total housing units,Total housing units,Total housing units,Occupied housing units,Occupied housing units,Occupied housing units,...,7 rooms,8 rooms,9 rooms or more,Median rooms,No bedroom,1 bedroom,2 bedrooms,3 bedrooms,4 bedrooms,5 or more bedrooms
0,2010,"Albany-Corvallis-Lebanon, OR","85,098","85,098","85,098","85,098","85,098","78,377","78,377","78,377",...,"11,633","6,693","8,135",5.5,"1,841","7,104","23,697","38,283","10,755","3,418"
1,2010,"Albany-Schenectady-Amsterdam, NY","545,444","545,444","545,444","545,444","545,444","470,055","470,055","470,055",...,"65,706","53,772","62,733",5.7,"8,420","64,685","145,350","209,224","92,675","25,090"
2,2010,"Ames-Boone, IA","48,557","48,557","48,557","48,557","48,557","46,302","46,302","46,302",...,"5,313","4,428","7,938",5.6,409,"5,851","13,942","17,362","8,813","2,180"
3,2010,"Appleton-Oshkosh-Neenah, WI","166,317","166,317","166,317","166,317","166,317","155,878","155,878","155,878",...,"20,396","14,463","20,271",5.6,"2,063","13,671","43,009","77,435","24,881","5,258"
4,2010,"Asheville-Brevard, NC","232,936","232,936","232,936","232,936","232,936","195,419","195,419","195,419",...,"27,185","17,411","21,233",5.4,"2,830","18,014","73,068","107,531","25,557","5,936"


In [9]:
income_files = [
    'acs-income/ACSST5Y2010.S1901-Data.csv',
    'acs-income/ACSST5Y2011.S1901-Data.csv',
    'acs-income/ACSST5Y2012.S1901-Data.csv',
    'acs-income/ACSST5Y2013.S1901-Data.csv',
    'acs-income/ACSST5Y2014.S1901-Data.csv',
    'acs-income/ACSST5Y2015.S1901-Data.csv',
    'acs-income/ACSST5Y2016.S1901-Data.csv',
    'acs-income/ACSST5Y2017.S1901-Data.csv',
    'acs-income/ACSST5Y2018.S1901-Data.csv',
    'acs-income/ACSST5Y2019.S1901-Data.csv',
    'acs-income/ACSST5Y2020.S1901-Data.csv',
    'acs-income/ACSST1Y2021.S1901-Data.csv',
]

income_df = pd.DataFrame()
for file_path in income_files:
    income_year_df = pd.read_csv(file_path, skiprows=1).set_index("Geographic Area Name")
    year = re.match(r'.*ACSST\dY(?P<year>\d{4}).*', file_path).group('year')
    
    # Make sure we name our income column correctly
    if 'Estimate!!Households!!Total' in income_year_df.columns:
        income_col = 'Estimate!!Households!!Total'
    else:
        income_col = 'Households!!Estimate!!Total'
    
    # Now just take the column we want
    income_year_df = income_year_df[[income_col]].copy()
    income_year_df.rename(columns={income_col: 'Income'}, inplace=True)
    
    # Remove unnecessary observations
    index = [idx for idx in income_year_df.index if idx.endswith(" Metro Area") or idx.endswith(" Micro Area")]
    income_year_df = income_year_df.loc[index]
    
    # Rename our indices to just be the CSA Name
    renamed_index = tuple(idx.replace(" Metro Area", "").replace(" Micro Area", "") for idx in income_year_df.index)
    income_year_df.index = renamed_index
        
    # Set index
    income_year_df.index.name = "MSA"
    income_year_df['Year'] = year
    income_year_df = income_year_df.reset_index().set_index(['Year', 'MSA'])
    
    # Look for intersection between our indices ...
    income_df = pd.concat([income_df, income_year_df]).copy()

income_df.reset_index(inplace=True)
income_df.head()

,Year,MSA,Income
0,2010,"Abbeville, LA",21786.0
1,2010,"Aberdeen, SD",16529.0
2,2010,"Aberdeen, WA",28191.0
3,2010,"Abilene, TX",60912.0
4,2010,"Ada, OK",14782.0


In [10]:
permits_df = pd.concat(msa_dfs).dropna(axis = 1)
permits_df.Name = permits_df.Name.apply(lambda x: x.strip())
permits_df['Year'] = permits_df.Date.apply(lambda x: int(x[:4]))
permits_df['Month'] = permits_df.Date.apply(lambda x: int(x[4:]))

permits_df.head()

,CSA,CBSA,Name,Total,1 Unit,2 Units,3 and 4 Units,5 Units or More,Num of Structures With 5 Units or More,Date,Year,Month
1,999.0,10180.0,"Abilene, TX",33.0,31.0,2.0,0.0,0.0,0.0,202102,2021,2
2,184.0,10420.0,"Akron, OH",45.0,45.0,0.0,0.0,0.0,0.0,202102,2021,2
3,999.0,10500.0,"Albany, GA",32.0,15.0,0.0,0.0,17.0,1.0,202102,2021,2
4,440.0,10540.0,"Albany-Lebanon, OR",58.0,37.0,0.0,0.0,21.0,3.0,202102,2021,2
5,104.0,10580.0,"Albany-Schenectady-Troy, NY",268.0,118.0,4.0,0.0,146.0,2.0,202102,2021,2


In [11]:
ZHVI_dfs = [pd.read_csv(z) for z in ZHVI_files]
ZHVI_dfs = [df[df['RegionType']=='msa'] for df in ZHVI_dfs]

ZHVI_dfs[0].head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,...,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31
1,394913,1,"New York, NY",msa,NY,398618.0,400989.0,403125.0,407346.0,411288.0,...,944392.0,949401.0,955581.0,963163.0,973716.0,985181.0,997357.0,1007381.0,1014851.0,1019097.0
2,753899,2,"Los Angeles, CA",msa,CA,412833.0,414349.0,416476.0,420539.0,424798.0,...,1459690.0,1475813.0,1494850.0,1516710.0,1547884.0,1582369.0,1612876.0,1617590.0,1615957.0,1595977.0
3,394463,3,"Chicago, IL",msa,IL,295109.0,295865.0,296827.0,298700.0,300646.0,...,464815.0,468835.0,472579.0,476171.0,481214.0,486852.0,493379.0,498309.0,501462.0,502214.0
4,394514,4,"Dallas, TX",msa,TX,231125.0,231200.0,231294.0,231544.0,231819.0,...,546482.0,556356.0,568739.0,582375.0,595125.0,610729.0,625328.0,638078.0,641222.0,639479.0
5,394692,5,"Houston, TX",msa,TX,226539.0,226746.0,226568.0,226884.0,226956.0,...,455285.0,460482.0,465816.0,472578.0,481148.0,490962.0,500094.0,506863.0,510726.0,512294.0


In [12]:
for df,f in zip(ZHVI_dfs, ZHVI_files):
    df['filename'] = f[f.rfind('/')+1:-4]
df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,...,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,filename
1,394913,1,"New York, NY",msa,NY,269152.0,270818.0,272236.0,275165.0,277939.0,...,653921.0,658777.0,664655.0,672580.0,681386.0,690621.0,698178.0,703790.0,706621.0,Metro_zhvi_bdrmcnt_4_uc_sfrcondo_tier_0.33_0.6...
2,753899,2,"Los Angeles, CA",msa,CA,286048.0,287357.0,288898.0,291929.0,294973.0,...,1035580.0,1052255.0,1070162.0,1094323.0,1119753.0,1142443.0,1145790.0,1142258.0,1125769.0,Metro_zhvi_bdrmcnt_4_uc_sfrcondo_tier_0.33_0.6...
3,394463,3,"Chicago, IL",msa,IL,216379.0,216954.0,217723.0,219175.0,220698.0,...,388057.0,391541.0,395013.0,399657.0,404845.0,410553.0,414859.0,417500.0,418126.0,Metro_zhvi_bdrmcnt_4_uc_sfrcondo_tier_0.33_0.6...
4,394514,4,"Dallas, TX",msa,TX,183601.0,183579.0,183670.0,183826.0,184014.0,...,445207.0,455516.0,466538.0,476764.0,489400.0,501440.0,511803.0,514244.0,512422.0,Metro_zhvi_bdrmcnt_4_uc_sfrcondo_tier_0.33_0.6...
5,394692,5,"Houston, TX",msa,TX,165886.0,165941.0,165744.0,165896.0,165865.0,...,338518.0,342701.0,348083.0,354884.0,362637.0,369808.0,375118.0,378198.0,379363.0,Metro_zhvi_bdrmcnt_4_uc_sfrcondo_tier_0.33_0.6...


In [13]:
ZHVI_df = pd.concat(ZHVI_dfs)

In [14]:
print(ZHVI_df.columns)

Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
       '2000-01-31', '2000-02-29', '2000-03-31', '2000-04-30', '2000-05-31',
       ...
       '1999-03-31', '1999-04-30', '1999-05-31', '1999-06-30', '1999-07-31',
       '1999-08-31', '1999-09-30', '1999-10-31', '1999-11-30', '1999-12-31'],
      dtype='object', length=326)


In [15]:
ZHVI_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,...,1999-03-31,1999-04-30,1999-05-31,1999-06-30,1999-07-31,1999-08-31,1999-09-30,1999-10-31,1999-11-30,1999-12-31
1,394913,1,"New York, NY",msa,NY,398618.0,400989.0,403125.0,407346.0,411288.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,753899,2,"Los Angeles, CA",msa,CA,412833.0,414349.0,416476.0,420539.0,424798.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,394463,3,"Chicago, IL",msa,IL,295109.0,295865.0,296827.0,298700.0,300646.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,394514,4,"Dallas, TX",msa,TX,231125.0,231200.0,231294.0,231544.0,231819.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,394692,5,"Houston, TX",msa,TX,226539.0,226746.0,226568.0,226884.0,226956.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
ZHVI_df = pd.concat(ZHVI_dfs)

date_columns = [column for column in ZHVI_df.columns if re.match(r'\d{4}-\d{2}-\d{2}', column)]
other_columns = [column for column in ZHVI_df.columns if not re.match(r'\d{4}-\d{2}-\d{2}', column)]
ZHVI_df = ZHVI_df.melt(id_vars=other_columns, value_vars=date_columns, var_name='Date', value_name='Price')

# ZHVI_df.dropna(subset=['Price',], axis='columns', inplace=True)
ZHVI_df['Year'] = ZHVI_df.Date.apply(lambda x: int(x.split("-")[0]))
ZHVI_df['Month'] = ZHVI_df.Date.apply(lambda x: int(x.split("-")[1]))

ZHVI_df.head(10)

,RegionID,SizeRank,RegionName,RegionType,StateName,filename,Date,Price,Year,Month
0,394913,1,"New York, NY",msa,NY,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,2000-01-31,398618.0,2000,1
1,753899,2,"Los Angeles, CA",msa,CA,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,2000-01-31,412833.0,2000,1
2,394463,3,"Chicago, IL",msa,IL,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,2000-01-31,295109.0,2000,1
3,394514,4,"Dallas, TX",msa,TX,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,2000-01-31,231125.0,2000,1
4,394692,5,"Houston, TX",msa,TX,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,2000-01-31,226539.0,2000,1
5,395209,6,"Washington, DC",msa,VA,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,2000-01-31,321180.0,2000,1
6,394856,7,"Miami, FL",msa,FL,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,2000-01-31,244156.0,2000,1
7,394974,8,"Philadelphia, PA",msa,PA,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,2000-01-31,224947.0,2000,1
8,394347,9,"Atlanta, GA",msa,GA,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,2000-01-31,258618.0,2000,1
9,394976,10,"Phoenix, AZ",msa,AZ,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,2000-01-31,234570.0,2000,1


In [17]:
# Match Zillow MSAs to Census Permit Data MSAs

# Find all unique values from each data set
def match_msas(out_file, match_df, match_col, zillow_df, zillow_col='RegionName'):
    zillow_msa_set = set(zillow_df[zillow_col].unique())
    match_msa_set = set(match_df[match_col].unique())

    # Get exact matches and mismatches
    msa_intersection = zillow_msa_set.intersection(match_msa_set)
    unmatched_zillow_msas = zillow_msa_set.difference(match_msa_set)
    unmatched_match_msas = match_msa_set.difference(zillow_msa_set)

    # Set of tuples of form (ZHVI msa, Census MSA)
    msa_matches = set()

    # Add our exact matches to our set of matches
    for matched_msa in msa_intersection:
        msa_matches.add((matched_msa, matched_msa))

    # Print out number of auto-matches
    print((f"Matched: {len(msa_matches)}, Unmatched Zillow: {len(unmatched_zillow_msas)}, "
           f"Unmatched Other MSAs {len(unmatched_match_msas)}"))

    # Continue matching by looking at city/state values in MSA names for imprecise matches
    left_matches = set()
    for uzm in unmatched_zillow_msas:
        city, state = uzm.split(", ")
        for match_msa in unmatched_match_msas:
            if city in match_msa and state in match_msa:
                left_matches.add((uzm, match_msa))
                break

    # Put our matches into a dataframe
    left_match_df = pd.DataFrame(left_matches, columns=['Zillow', 'MSA'])
    auto_match_df = pd.DataFrame(msa_matches, columns=['Zillow', 'MSA'])

    # Save our matches to a file
    msa_match_df = pd.concat([left_match_df, auto_match_df]).sort_values(['MSA', 'Zillow'])
    msa_match_df.to_csv(out_file, index=False)
    return msa_match_df

permit_match_df = match_msas('permit_msa_match.csv', match_df=permits_df, match_col='Name', zillow_df=ZHVI_df)
permit_match_df.head()

Matched: 216, Unmatched Zillow: 682, Unmatched Other MSAs 168


,Zillow,MSA
80,"Abilene, TX","Abilene, TX"
181,"Akron, OH","Akron, OH"
169,"Albany, GA","Albany, GA"
47,"Albany, OR","Albany-Lebanon, OR"
78,"Albany, NY","Albany-Schenectady-Troy, NY"


In [18]:
income_match_df = match_msas('income_msa_match.csv', match_df=income_df, match_col='MSA', zillow_df=ZHVI_df)
income_match_df.head()

Matched: 717, Unmatched Zillow: 181, Unmatched Other MSAs 416


,Zillow,MSA
262,"Aberdeen, SD","Aberdeen, SD"
496,"Aberdeen, WA","Aberdeen, WA"
244,"Abilene, TX","Abilene, TX"
681,"Ada, OK","Ada, OK"
90,"Adrian, MI","Adrian, MI"


In [19]:
housing_match_df = match_msas('housing_msa_match.csv', match_df=housing_df, match_col='CSA', zillow_df=ZHVI_df)
housing_match_df.head()

Matched: 0, Unmatched Zillow: 898, Unmatched Other MSAs 301


,Zillow,MSA
133,"Albany, OR","Albany-Corvallis-Lebanon, OR"
333,"Corvallis, OR","Albany-Corvallis-Lebanon, OR"
257,"Albany, NY","Albany-Schenectady, NY"
238,"Amsterdam, NY","Albany-Schenectady-Amsterdam, NY"
226,"Albuquerque, NM","Albuquerque-Santa Fe-Las Vegas, NM"


In [20]:
big_df = permit_match_df.merge(permits_df, how='left', left_on='MSA', right_on='Name')
big_df = big_df.merge(
    ZHVI_df, how='inner', left_on=['Zillow', 'Year', 'Month'], right_on=['RegionName', 'Year', 'Month']
)

drop_columns = ['Name', 'RegionID', 'RegionName', 'RegionType',  'SizeRank', 'Date_x', 'Date_y', 'StateName']
big_df['Date'] = pd.to_datetime(big_df['Date_y'])

big_df.drop(columns=drop_columns, inplace=True)
big_df.head()

,Zillow,MSA,CSA,CBSA,Total,1 Unit,2 Units,3 and 4 Units,5 Units or More,Num of Structures With 5 Units or More,Year,Month,filename,Price,Date
0,"Abilene, TX","Abilene, TX",999.0,10180.0,33.0,31.0,2.0,0.0,0.0,0.0,2021,2,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,254596.0,2021-02-28
1,"Abilene, TX","Abilene, TX",999.0,10180.0,33.0,31.0,2.0,0.0,0.0,0.0,2021,2,Metro_zhvi_bdrmcnt_2_uc_sfrcondo_tier_0.33_0.6...,99806.0,2021-02-28
2,"Abilene, TX","Abilene, TX",999.0,10180.0,33.0,31.0,2.0,0.0,0.0,0.0,2021,2,Metro_zhvi_uc_condo_tier_0.33_0.67_sm_sa_month,105050.0,2021-02-28
3,"Abilene, TX","Abilene, TX",999.0,10180.0,33.0,31.0,2.0,0.0,0.0,0.0,2021,2,Metro_zhvi_bdrmcnt_3_uc_sfrcondo_tier_0.33_0.6...,166200.0,2021-02-28
4,"Abilene, TX","Abilene, TX",999.0,10180.0,33.0,31.0,2.0,0.0,0.0,0.0,2021,2,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...,90655.0,2021-02-28


In [21]:
# big_df.shape
big_df.dtypes

Zillow                                            object
MSA                                               object
CSA                                              float64
CBSA                                             float64
Total                                            float64
1 Unit                                           float64
2 Units                                          float64
3 and 4 Units                                    float64
5 Units or More                                  float64
Num of Structures With 5 Units or More           float64
Year                                               int64
Month                                              int64
filename                                          object
Price                                            float64
Date                                      datetime64[ns]
dtype: object

In [22]:
big_df.head()

,Zillow,MSA,CSA,CBSA,Total,1 Unit,2 Units,3 and 4 Units,5 Units or More,Num of Structures With 5 Units or More,Year,Month,filename,Price,Date
0,"Abilene, TX","Abilene, TX",999.0,10180.0,33.0,31.0,2.0,0.0,0.0,0.0,2021,2,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,254596.0,2021-02-28
1,"Abilene, TX","Abilene, TX",999.0,10180.0,33.0,31.0,2.0,0.0,0.0,0.0,2021,2,Metro_zhvi_bdrmcnt_2_uc_sfrcondo_tier_0.33_0.6...,99806.0,2021-02-28
2,"Abilene, TX","Abilene, TX",999.0,10180.0,33.0,31.0,2.0,0.0,0.0,0.0,2021,2,Metro_zhvi_uc_condo_tier_0.33_0.67_sm_sa_month,105050.0,2021-02-28
3,"Abilene, TX","Abilene, TX",999.0,10180.0,33.0,31.0,2.0,0.0,0.0,0.0,2021,2,Metro_zhvi_bdrmcnt_3_uc_sfrcondo_tier_0.33_0.6...,166200.0,2021-02-28
4,"Abilene, TX","Abilene, TX",999.0,10180.0,33.0,31.0,2.0,0.0,0.0,0.0,2021,2,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...,90655.0,2021-02-28


In [23]:
trend_df = pd.DataFrame()

for index, seasonal_df in big_df.set_index('Date').sort_index().groupby(['filename', 'Zillow']):
    try:
        decompose_result = seasonal_decompose(seasonal_df.Price.dropna())
        
        seasonal_df = seasonal_df.join(decompose_result.seasonal)
        seasonal_df = seasonal_df.join(decompose_result.trend)
        seasonal_df = seasonal_df.join(decompose_result.resid)
        
        seasonal_df.rename(columns={'seasonal': 'Seasonal', 'trend': 'Trend', 'resid': 'Residual'}, inplace=True)
    except ValueError:
        seasonal_df['Seasonal'] = None
        seasonal_df['Trend'] = None
        seasonal_df['Residual'] = None
    
    seasonal_df['Price Change'] = seasonal_df.Price.diff(periods=1)
    
    try:
        seasonal_df['Trend Change'] = seasonal_df.Trend.diff(periods=1)
    except TypeError:
        seasonal_df['Trend Change'] = None
    
    seasonal_df.reset_index(inplace=True)
    trend_df = pd.concat([trend_df, seasonal_df])

trend_df.head()

,Date,Zillow,MSA,CSA,CBSA,Total,1 Unit,2 Units,3 and 4 Units,5 Units or More,Num of Structures With 5 Units or More,Year,Month,filename,Price,Seasonal,Trend,Residual,Price Change,Trend Change
0,2019-11-30,"Abilene, TX","Abilene, TX",999.0,10180.0,23.0,21.0,2.0,0.0,0.0,0.0,2019,11,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...,81284.0,1232.322917,NaN,NaN,NaN,NaN
1,2019-12-31,"Abilene, TX","Abilene, TX",999.0,10180.0,37.0,31.0,6.0,0.0,0.0,0.0,2019,12,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...,81875.0,869.385417,NaN,NaN,591.0,NaN
2,2020-01-31,"Abilene, TX","Abilene, TX",999.0,10180.0,29.0,29.0,0.0,0.0,0.0,0.0,2020,1,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...,82268.0,533.177083,NaN,NaN,393.0,NaN
3,2020-02-29,"Abilene, TX","Abilene, TX",999.0,10180.0,30.0,30.0,0.0,0.0,0.0,0.0,2020,2,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...,82846.0,-645.781250,NaN,NaN,578.0,NaN
4,2020-03-31,"Abilene, TX","Abilene, TX",999.0,10180.0,29.0,27.0,2.0,0.0,0.0,0.0,2020,3,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...,84280.0,-1380.489583,NaN,NaN,1434.0,NaN


In [24]:
seasonal_df[-10:]

,Date,Zillow,MSA,CSA,CBSA,Total,1 Unit,2 Units,3 and 4 Units,5 Units or More,Num of Structures With 5 Units or More,Year,Month,filename,Price,Seasonal,Trend,Residual,Price Change,Trend Change
23,2021-10-31,"Yuma, AZ","Yuma, AZ",999.0,49740.0,71.0,71.0,0.0,0.0,0.0,0.0,2021,10,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,328588.0,403.279514,326745.125000,1439.595486,6781.0,6074.833333
24,2021-11-30,"Yuma, AZ","Yuma, AZ",999.0,49740.0,48.0,48.0,0.0,0.0,0.0,0.0,2021,11,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,333382.0,26.092014,332935.208333,420.699653,4794.0,6190.083333
25,2021-12-31,"Yuma, AZ","Yuma, AZ",999.0,49740.0,51.0,51.0,0.0,0.0,0.0,0.0,2021,12,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,339164.0,370.675347,339246.000000,-452.675347,5782.0,6310.791667
26,2022-01-31,"Yuma, AZ","Yuma, AZ",999.0,49740.0,72.0,72.0,0.0,0.0,0.0,0.0,2022,1,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,345701.0,877.654514,345459.625000,-636.279514,6537.0,6213.625000
27,2022-02-28,"Yuma, AZ","Yuma, AZ",999.0,49740.0,91.0,87.0,4.0,0.0,0.0,0.0,2022,2,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,351573.0,1011.050347,NaN,NaN,5872.0,NaN
28,2022-03-31,"Yuma, AZ","Yuma, AZ",999.0,49740.0,89.0,83.0,6.0,0.0,0.0,0.0,2022,3,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,357302.0,954.425347,NaN,NaN,5729.0,NaN
29,2022-04-30,"Yuma, AZ","Yuma, AZ",999.0,49740.0,66.0,66.0,0.0,0.0,0.0,0.0,2022,4,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,363285.0,684.842014,NaN,NaN,5983.0,NaN
30,2022-05-31,"Yuma, AZ","Yuma, AZ",999.0,49740.0,88.0,84.0,4.0,0.0,0.0,0.0,2022,5,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,370578.0,-993.157986,NaN,NaN,7293.0,NaN
31,2022-06-30,"Yuma, AZ","Yuma, AZ",999.0,49740.0,65.0,65.0,0.0,0.0,0.0,0.0,2022,6,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,376555.0,-932.491319,NaN,NaN,5977.0,NaN
32,2022-07-31,"Yuma, AZ","Yuma, AZ",999.0,49740.0,171.0,81.0,0.0,0.0,90.0,6.0,2022,7,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,379967.0,-1460.053819,NaN,NaN,3412.0,NaN


In [25]:
trend_df[trend_df.MSA == 'Abilene, TX'].head(10)

,Date,Zillow,MSA,CSA,CBSA,Total,1 Unit,2 Units,3 and 4 Units,5 Units or More,Num of Structures With 5 Units or More,Year,Month,filename,Price,Seasonal,Trend,Residual,Price Change,Trend Change
0,2019-11-30,"Abilene, TX","Abilene, TX",999.0,10180.0,23.0,21.0,2.0,0.0,0.0,0.0,2019,11,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...,81284.0,1232.322917,NaN,NaN,NaN,NaN
1,2019-12-31,"Abilene, TX","Abilene, TX",999.0,10180.0,37.0,31.0,6.0,0.0,0.0,0.0,2019,12,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...,81875.0,869.385417,NaN,NaN,591.0,NaN
2,2020-01-31,"Abilene, TX","Abilene, TX",999.0,10180.0,29.0,29.0,0.0,0.0,0.0,0.0,2020,1,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...,82268.0,533.177083,NaN,NaN,393.0,NaN
3,2020-02-29,"Abilene, TX","Abilene, TX",999.0,10180.0,30.0,30.0,0.0,0.0,0.0,0.0,2020,2,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...,82846.0,-645.781250,NaN,NaN,578.0,NaN
4,2020-03-31,"Abilene, TX","Abilene, TX",999.0,10180.0,29.0,27.0,2.0,0.0,0.0,0.0,2020,3,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...,84280.0,-1380.489583,NaN,NaN,1434.0,NaN
5,2020-04-30,"Abilene, TX","Abilene, TX",999.0,10180.0,29.0,27.0,2.0,0.0,0.0,0.0,2020,4,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...,85200.0,-2239.614583,NaN,NaN,920.0,NaN
6,2020-05-31,"Abilene, TX","Abilene, TX",999.0,10180.0,20.0,20.0,0.0,0.0,0.0,0.0,2020,5,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...,85497.0,-331.572917,84621.833333,1206.739583,297.0,NaN
7,2020-06-30,"Abilene, TX","Abilene, TX",999.0,10180.0,201.0,40.0,2.0,20.0,139.0,17.0,2020,6,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...,84757.0,-887.718750,85266.958333,377.760417,-740.0,645.125000
8,2020-07-31,"Abilene, TX","Abilene, TX",999.0,10180.0,113.0,42.0,14.0,0.0,57.0,2.0,2020,7,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...,84338.0,-163.718750,85907.125000,-1405.406250,-419.0,640.166667
9,2020-08-31,"Abilene, TX","Abilene, TX",999.0,10180.0,33.0,31.0,2.0,0.0,0.0,0.0,2020,8,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...,84990.0,462.197917,86554.041667,-2026.239583,652.0,646.916667


In [26]:
def get_pop(x):
    if np.isnan(x['Population 1']) and np.isnan(x['Population 5']):
        return None
    
    if np.isnan(x['Population 1']):
        return x['Population 5']
    if np.isnan(x['Population 5']):
        return x['Population 1']
    
    return (x['Population 5'] + x['Population 5'])/2

In [27]:
acs_1_df = pd.DataFrame()
# Add ACS 1-year estimates
for year in (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021):
    acs_year_df = pd.read_csv(f"acs-1-year/ACSDP1Y{year}.DP05-Data.csv", skiprows=1)[1:].copy()
    acs_year_df['MSA'] = acs_year_df['Geographic Area Name'].apply(
        lambda x: x.replace(" Micro Area", "").replace(" Metro Area", "")
    )
    
    acs_year_df.rename(columns={'Estimate!!SEX AND AGE!!Total population': 'Population 1'}, inplace=True)
    acs_year_df = acs_year_df[["MSA", "Population 1"]].copy()
    acs_year_df['Year'] = year
    
    acs_1_df = pd.concat([acs_1_df, acs_year_df])

# Add ACS 5-year estimates
acs_5_df = pd.DataFrame()
for year in (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020):
    acs_year_df = pd.read_csv(f"acs-5-year/ACSDP5Y{year}.DP05-Data.csv", skiprows=1)[1:].copy()
    acs_year_df['MSA'] = acs_year_df['Geographic Area Name'].apply(
        lambda x: x.replace(" Micro Area", "").replace(" Metro Area", "")
    )
    
    acs_year_df.rename(columns={'Estimate!!SEX AND AGE!!Total population': 'Population 5'}, inplace=True)
    acs_year_df = acs_year_df[["MSA", "Population 5"]].copy()
    acs_year_df['Year'] = year
    
    acs_5_df = pd.concat([acs_5_df, acs_year_df])

# Merge our datasets and reconcile our estimates
acs_df = pd.merge(acs_1_df, acs_5_df, on=['MSA', 'Year'], how='outer')
acs_df['Population Diff'] = 1 - acs_df['Population 5'] / acs_df['Population 1']
acs_df['Population'] = acs_df.apply(get_pop, axis=1)


# Add population growth as features
def add_pop_growth(df, diff):
    df['Year as Date'] = df.Year.apply(lambda year: dt.datetime(year=year, month=1, day=1))
    df = df.set_index(['MSA', 'Year as Date']).sort_index()
    
    df[f'Pop Growth {diff} Year'] = df.groupby('MSA')['Population'].diff(diff)
    df[f'Pop -{diff} Years'] = df.groupby('MSA')['Population'].shift(diff)
    df[f'Pop Percent {diff} Year'] = df[f'Pop Growth {diff} Year'] / df[f'Pop -{diff} Years']
    
    df.reset_index(inplace=True)
    return df


acs_df = add_pop_growth(acs_df, 1)
acs_df = add_pop_growth(acs_df, 5)
acs_df = acs_df[acs_df.Year >= 2019].copy()

acs_df.head()

,MSA,Year as Date,Population 1,Year,Population 5,Population Diff,Population,Pop Growth 1 Year,Pop -1 Years,Pop Percent 1 Year,Pop Growth 5 Year,Pop -5 Years,Pop Percent 5 Year
12,"Aberdeen, SD",2019-01-01,NaN,2019,42824.0,NaN,42824.0,44.0,42780.0,0.001029,1260.0,41564.0,0.030315
13,"Aberdeen, SD",2020-01-01,NaN,2020,42864.0,NaN,42864.0,40.0,42824.0,0.000934,786.0,42078.0,0.018680
23,"Aberdeen, WA",2019-01-01,75061.0,2019,72779.0,0.030402,72779.0,812.0,71967.0,0.011283,1045.0,71734.0,0.014568
24,"Aberdeen, WA",2020-01-01,NaN,2020,73769.0,NaN,73769.0,990.0,72779.0,0.013603,2350.0,71419.0,0.032904
25,"Aberdeen, WA",2021-01-01,76841.0,2021,NaN,NaN,76841.0,3072.0,73769.0,0.041644,5608.0,71233.0,0.078728


In [28]:
acs_df[acs_df.MSA == 'Abilene, TX']

,MSA,Year as Date,Population 1,Year,Population 5,Population Diff,Population,Pop Growth 1 Year,Pop -1 Years,Pop Percent 1 Year,Pop Growth 5 Year,Pop -5 Years,Pop Percent 5 Year
35,"Abilene, TX",2019-01-01,171795.0,2019,170669.0,0.006554,170669.0,660.0,170009.0,0.003882,3498.0,167171.0,0.020925
36,"Abilene, TX",2020-01-01,NaN,2020,171354.0,NaN,171354.0,685.0,170669.0,0.004014,3409.0,167945.0,0.020298
37,"Abilene, TX",2021-01-01,178608.0,2021,NaN,NaN,178608.0,7254.0,171354.0,0.042333,9834.0,168774.0,0.058267


In [29]:
acs_df.to_csv("ACS.csv", index=False)

In [30]:
combined_df = pd.merge(trend_df, acs_df, on=['MSA', 'Year'], how='left')
combined_df.to_csv("combined.csv")

combined_df.head()

,Date,Zillow,MSA,CSA,CBSA,Total,1 Unit,2 Units,3 and 4 Units,5 Units or More,...,Population 1,Population 5,Population Diff,Population,Pop Growth 1 Year,Pop -1 Years,Pop Percent 1 Year,Pop Growth 5 Year,Pop -5 Years,Pop Percent 5 Year
0,2019-11-30,"Abilene, TX","Abilene, TX",999.0,10180.0,23.0,21.0,2.0,0.0,0.0,...,171795.0,170669.0,0.006554,170669.0,660.0,170009.0,0.003882,3498.0,167171.0,0.020925
1,2019-12-31,"Abilene, TX","Abilene, TX",999.0,10180.0,37.0,31.0,6.0,0.0,0.0,...,171795.0,170669.0,0.006554,170669.0,660.0,170009.0,0.003882,3498.0,167171.0,0.020925
2,2020-01-31,"Abilene, TX","Abilene, TX",999.0,10180.0,29.0,29.0,0.0,0.0,0.0,...,NaN,171354.0,NaN,171354.0,685.0,170669.0,0.004014,3409.0,167945.0,0.020298
3,2020-02-29,"Abilene, TX","Abilene, TX",999.0,10180.0,30.0,30.0,0.0,0.0,0.0,...,NaN,171354.0,NaN,171354.0,685.0,170669.0,0.004014,3409.0,167945.0,0.020298
4,2020-03-31,"Abilene, TX","Abilene, TX",999.0,10180.0,29.0,27.0,2.0,0.0,0.0,...,NaN,171354.0,NaN,171354.0,685.0,170669.0,0.004014,3409.0,167945.0,0.020298


In [31]:
combined_df.dropna(subset=['Population']).shape

(94718, 31)

In [32]:
# 5 year: 2019, 2020
# 1 year: 2019, 2021

In [33]:
housing_df['MSA'] = housing_df['CSA']

housing_df.Year = housing_df.Year.astype('int64')
income_df.Year = income_df.Year.astype('int64')

In [34]:
combo_with_h_i_df = pd.merge(combined_df, housing_df, on = ['MSA', 'Year'], how = 'outer')
combo_with_h_i_df = pd.merge(combo_with_h_i_df, income_df, on = ['MSA', 'Year'], how = 'outer')

In [35]:

combo_with_h_i_df = combo_with_h_i_df[['Date', 'MSA','Total', '1 Unit', '2 Units', '3 and 4 Units', 
                                       '5 Units or More', 'Num of Structures With 5 Units or More',
                                       'Month', 'filename', 'Price', 'Seasonal', 'Trend', 'Residual', 'Price Change', 
                                       'Trend Change', 'Year as Date', 'Population 1', 'Population 5', 
                                       'Population Diff', 'Population', 'Pop Growth 1 Year', 'Pop -1 Years', 
                                       'Pop Percent 1 Year', 'Pop Growth 5 Year', 'Pop -5 Years', 
                                       'Pop Percent 5 Year','Total housing units', 
                                       'Total housing units', 'Total housing units', 'Total housing units', 
                                       'Total housing units', 'Occupied housing units', 'Occupied housing units', 
                                       'Occupied housing units', 'Occupied housing units', 'Occupied housing units', 
                                       'Occupied housing units', 'Occupied housing units', 'Vacant housing units', 
                                       'Homeowner vacancy rate', 'Rental vacancy rate', 'Total housing units',
                                       'Total housing units', 'Total housing units', 'Total housing units', 
                                       'Total housing units', '1-unit, detached', '1-unit, attached', '2 units', 
                                       '3 or 4 units', '5 to 9 units', '10 to 19 units', '20 or more units', 
                                       '1 room', '2 rooms', '3 rooms', '4 rooms', '5 rooms', '6 rooms', '7 rooms', 
                                       '8 rooms', '9 rooms or more', 'Median rooms', 'No bedroom', '1 bedroom', 
                                       '2 bedrooms', '3 bedrooms', '4 bedrooms', '5 or more bedrooms', 'Year', 
                                       'Income']]


In [36]:
combo_with_h_i_df.to_csv('combo_housing_income.csv')